In [24]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
import itertools
import xgboost as xgb

In [25]:
train = pd.read_parquet('../input/train_fe.parquet')
test = pd.read_parquet('../input/test_fe.parquet')

In [26]:
train = train[['customer_ID', 'P_2_last', 'D_39_last', 'target']] #, 'P_2_last', 'D_39_last'
test = test[['customer_ID', 'P_2_last', 'D_39_last']]

In [27]:
test.head()

,customer_ID,P_2_last,D_39_last
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.568930,4
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.841177,4
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.697522,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.513186,11
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.254478,26


In [28]:
# Metric:

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# XGB amex metric
# ====================================================
def xgb_amex(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred)

In [29]:
oof_new = pd.read_csv('../lgbm_new_5_fold/oof_lgbm_new_5_fold_seed42.csv')
oof_smn = pd.read_csv('../lgbm_smn_fold/oof_lgbm_smn_fold_seed42.csv')
oof_lag = pd.read_csv('../lgbm_lag_5_fold/oof_lgbm_lag_5_fold_5fold_seed42.csv')
oof_xgb = pd.read_csv('../xgb_fe_8_fold/oof_xgb_fe_8_fold_seed42.csv')
oof_xgb_smn = pd.read_csv('../xgb_smn_5_fold/oof_xgb_smn_5_fold_seed42.csv')
oof_bobw = pd.read_csv('../xgb_BOBW_5_fold/oof_xgb_BOBW_5_fold_seed42.csv')

In [30]:
oof_bobw.head()

,customer_ID,target,prediction
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.000272
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.000842
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.001372
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.007334
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.001293


In [31]:
train['pred_new'] = oof_new.prediction
train['pred_smn'] = oof_smn.prediction
train['pred_lag'] = oof_lag.prediction
train['pred_xgb'] = oof_xgb.prediction
train['pred_xgb_smn'] = oof_xgb_smn.prediction
train['pred_xgb_bobw'] = oof_bobw.prediction

In [32]:
train = train[['customer_ID' , 'P_2_last', 'D_39_last', 'pred_new', 'pred_smn', 'pred_lag', 'pred_xgb','pred_xgb_smn', 'pred_xgb_bobw', 'target']] #'pred_lag',

In [33]:
train.head()

,customer_ID,P_2_last,D_39_last,pred_new,pred_smn,pred_lag,pred_xgb,pred_xgb_smn,pred_xgb_bobw,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.934745,0,0.000307,0.000128,0.000346,0.000262,0.000162,0.000272,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.880519,6,0.000957,0.000704,0.000897,0.001429,0.001026,0.000842,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.880875,0,0.001072,0.001070,0.001014,0.001446,0.001510,0.001372,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.621776,0,0.003317,0.001992,0.003068,0.008611,0.003684,0.007334,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.871900,0,0.001673,0.000634,0.001234,0.000680,0.001328,0.001293,0


In [34]:
test_new = pd.read_csv('../lgbm_new_5_fold/test_lgbm_new_5_fold_seed42.csv')
test_smn = pd.read_csv('../lgbm_smn_fold/test_lgbm_smn_fold_seed42.csv')
test_lag = pd.read_csv('../lgbm_lag_5_fold/test_lgbm_lag_5_fold_5fold_seed42.csv')
test_xgb = pd.read_csv('../xgb_fe_8_fold/test_xgb_fe_8_fold_seed42.csv')
test_xgb_smn = pd.read_csv('../xgb_smn_5_fold/test_xgb_smn_5_fold_seed42.csv')
test_xgb_bobw = pd.read_csv('../xgb_BOBW_5_fold/test_xgb_BOBW_5_fold_seed42.csv')

In [35]:
test['pred_new'] = test_new.prediction
test['pred_smn'] = test_smn.prediction
test['pred_lag'] = test_lag.prediction
test['pred_xgb'] = test_xgb.prediction
test['pred_xgb_smn'] = test_xgb_smn.prediction
test['pred_xgb_bobw'] = test_xgb_bobw.prediction

In [36]:
from scipy.stats import ks_2samp

#perform Kolmogorov-Smirnov test
ks_2samp(test_xgb.prediction, test_new.prediction)

KstestResult(statistic=0.057840996473149525, pvalue=0.0)

In [37]:
test.head()

,customer_ID,P_2_last,D_39_last,pred_new,pred_smn,pred_lag,pred_xgb,pred_xgb_smn,pred_xgb_bobw
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.568930,4,0.029524,0.028134,0.029705,0.032877,0.024319,0.020313
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.841177,4,0.001012,0.000574,0.000823,0.000531,0.000385,0.000391
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.697522,0,0.038608,0.025484,0.044923,0.033457,0.027844,0.030673
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.513186,11,0.179223,0.200430,0.226944,0.153475,0.156202,0.126363
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.254478,26,0.869733,0.905022,0.837129,0.920015,0.925976,0.921779


In [38]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../input/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [39]:
# ====================================================
# Directory settings
# ====================================================
import os

expt_name = "stacking_xgb_2"
OUTPUT_DIR = f'../{expt_name}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [40]:
def xgb_train(x_train, y_train, x_val, y_val):
    print("# of features:", x_train.shape[1])
    assert x_train.shape[1] == x_val.shape[1]
    dtrain = xgb.DMatrix(data=x_train, label=y_train)
    dvalid = xgb.DMatrix(data=x_val, label=y_val)
    params = {
            'booster': 'dart',
            'rate_drop': 0.1,
            'skip_drop': 0.5,
            'objective': 'binary:logistic',
            'tree_method': 'gpu_hist',
            'max_depth': 7,
            'subsample':0.88, #0.88
            'colsample_bytree': 0.5,
            'gamma':1.5,
            'min_child_weight':8,
            'lambda':70,
            'eta':0.03, #0.03
    }
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain=dtrain,
                num_boost_round=2600,evals=watchlist,
                early_stopping_rounds=1000, feval=xgb_amex, maximize=True,
                verbose_eval=100)   #num_boost_round=2600, early_stopping_rounds = 500 , feval=xgb_amex
    print('best tree_limit:', model.best_ntree_limit)
    print('best score:', model.best_score)
    return model.predict(dvalid, iteration_range=(0,model.best_ntree_limit)), model
seed_everything(CFG.seed)

In [41]:
# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
test_data = test[features]
dtest = xgb.DMatrix(data=test_data)

In [42]:
# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = 42)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]

    # x = x_train, y = y_train, xt = x_val, yt = y_val

    val_pred, model = xgb_train(x_train, y_train, x_val, y_val)
    model.save_model(OUTPUT_DIR+f'{expt_name}_fold{fold}.xgb')
    # Add to out of folds array
    oof_predictions[val_ind] = val_pred

    # Predict the test set
    test_pred = model.predict(dtest)
    test_predictions += test_pred / CFG.n_folds

    amex_score = amex_metric(y_val, val_pred)

    print(f'Fold {fold} CV score is {amex_score}')

# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(OUTPUT_DIR+f'oof_{expt_name}_seed{CFG.seed}.csv', index = False)
# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
test_df.to_csv(OUTPUT_DIR+f'test_{expt_name}_seed{CFG.seed}.csv', index = False)

 
--------------------------------------------------
Training fold 0 with 8 features...
# of features: 8
[0]	train-logloss:0.67163	train-amex_metric:0.79296	eval-logloss:0.67153	eval-amex_metric:0.79866
[100]	train-logloss:0.24253	train-amex_metric:0.79792	eval-logloss:0.24088	eval-amex_metric:0.80331
[200]	train-logloss:0.21907	train-amex_metric:0.79848	eval-logloss:0.21732	eval-amex_metric:0.80367
[300]	train-logloss:0.21415	train-amex_metric:0.79865	eval-logloss:0.21248	eval-amex_metric:0.80362
[400]	train-logloss:0.21311	train-amex_metric:0.79866	eval-logloss:0.21156	eval-amex_metric:0.80359
[500]	train-logloss:0.21277	train-amex_metric:0.79876	eval-logloss:0.21137	eval-amex_metric:0.80367
[600]	train-logloss:0.21264	train-amex_metric:0.79898	eval-logloss:0.21134	eval-amex_metric:0.80358
[700]	train-logloss:0.21256	train-amex_metric:0.79912	eval-logloss:0.21134	eval-amex_metric:0.80356
[800]	train-logloss:0.21249	train-amex_metric:0.79914	eval-logloss:0.21134	eval-amex_metric:0.803

In [43]:
test_df.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.024639
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000745
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.036576
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.179826
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.890262


In [44]:
amex_metric(oof_df[CFG.target], oof_df.prediction)

0.7972298416722128